### Importing Packages

In [1]:
import itertools
import numpy as np
from Bio import Align
from Bio import SeqIO
from Bio import pairwise2
from Bio.SubsMat.MatrixInfo import blosum62
import csv
import pandas as pd
from Bio.pairwise2 import format_alignment
import math


### Reading fast files of protein sequence

In [3]:
import Bio

In [3]:

query_seq = list(SeqIO.parse("../Data/NpInter2 Interacting Protiens/NpInterV2_Interacting_Protiens_26_1923.fasta", "fasta"))
target_seq = list(SeqIO.parse("../Data/NpInter2 Interacting Protiens/NpInterV2_Interacting_Protiens_26_1923.fasta", "fasta"))
#query_seq = list(SeqIO.parse("../Data/NpInter2 Interacting Protiens/sample1.fasta", "fasta"))
#target_seq = list(SeqIO.parse("../Data/NpInter2 Interacting Protiens/sample1.fasta", "fasta"))


In [4]:
print(len(target_seq))
print(len(target_seq))

25
25


### custom format_alignment to get the match, mismatch, gaps and score

In [5]:
def fn_format_alignment(align1, align2, score, begin, end): 
    #s = [] 
    #s.append("%s\n" % align1) 
    #s.append(" " * begin) 
    #matrix=blosum62
    gap=0
    mismatch=0
    match=0
    seq_len=0
    mismatch_positive=0
    #print(align1[begin:end])
    #print(align2[begin:end])
    for a, b in zip(align1[begin:end], align2[begin:end]): 
        if a == b: 
            #s.append("|")  # match 
            match+=1
        elif a == "-" or b == "-":
            
            #s.append(" ")  # gap
            gap+=1
        elif a != b:
            if (a,b) in blosum62:
                if blosum62[(a,b)] >0:
                    mismatch_positive+=1
            else:
                if blosum62[(b,a)] >0:
                    mismatch_positive+=1
            
                                                     
            #s.append(".")  # mismatch 
            mismatch+=1
        seq_len = min(len(align1[begin:end]), len(align2[begin:end]))
    #s.append("\n") 
    #s.append("%s\n" % align2) 
    #s.append("  Score=%g\n" % score)
    #s.append("  Gap=%g\n" % gap)
    #s.append("  Mismatch=%g\n" % mismatch)
    return score,match,mismatch,gap,seq_len,mismatch_positive

In [6]:
# round decimal number

import decimal

def round_down(value, decimals):
    with decimal.localcontext() as ctx:
        d = decimal.Decimal(value)
        ctx.rounding = decimal.ROUND_DOWN
        return round(d, decimals)

### Creating Empty CSV file to store the similarity results

#### Calculating similarity score and saving in csv

In [9]:

def calculate_similarity(query_seq,target_seq):
    print("started task...")
    with open("../Data/NpInter2 Interacting Protiens/Protein-Protein Sequence Similarity/Protien-Protien_Similarity_26_1923.csv", "w", newline='') as csvFile:
        # csv file header
        fieldnames = ['Query_Seq_ID','Target_Seq_ID', 'Align_Score','Normalize_Score', 'Identity', 'Similarity']
        writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
        writer.writeheader()
        # loop on query seq
        count=0
        for seq_q in query_seq:
            try:
                # self alignment with query seq
                seq_q_align = pairwise2.align.localds(seq_q.seq, seq_q.seq, blosum62, -10, -0.5, one_alignment_only=True)
                # loop on target seq
                for seq_t in target_seq:
                    #if seq_q.seq != seq_t.seq:
                        
                    try:
                        # self alignment with traget seq
                        seq_t_align = pairwise2.align.localds(seq_t.seq, seq_t.seq, blosum62, -10, -0.5, one_alignment_only=True)
                        # getting the score of query seq self align
                        seq_q_align_score=fn_format_alignment(*seq_q_align[0])[0]
                        # getting the score of target seq self align
                        seq_t_align_score=fn_format_alignment(*seq_t_align[0])[0]


                        # the alignment b/w qry and trg seq
                        align = pairwise2.align.localds(seq_q.seq, seq_t.seq, blosum62, -10, -0.5, one_alignment_only=True)
                        # getting the score of query and target seq
                        score =fn_format_alignment(*align[0])[0]
                        # normalizing the score
                        Normalize_Score = round_down(score/(math.sqrt(seq_q_align_score)*math.sqrt(seq_t_align_score)),2)
                        # get the no of matches in query and target seq
                        matches = fn_format_alignment(*align[0])[1]
                        # getting the no of mismatches in qry and trg seq
                        mismatches = fn_format_alignment(*align[0])[2]
                        mismatch_positive=fn_format_alignment(*align[0])[5]
                        # calcualting the percent identity %.
                        seq_len = fn_format_alignment(*align[0])[4]
                        Identity = str(round_down(matches/seq_len * 100,2)) +'%' 

                        #Identity =str(round_down(matches/(matches + mismatches) * 100,2)) +'%' 
                        # calculating the similarity in %
                        #Similarity=str(round_down((matches + mismatches)/seq_len * 100,2)) + '%'
                        Similarity=str(round_down((matches + mismatch_positive)/seq_len * 100,2)) + '%'
                        writer.writerow({'Query_Seq_ID': seq_q.id.split('|')[1],                                      
                                         'Target_Seq_ID': seq_t.id.split('|')[1],
                                         'Align_Score': score, 'Normalize_Score':Normalize_Score,
                                         'Identity':Identity, 'Similarity':Similarity })
                        count +=1
                        print(count)

                        print(seq_q.id.split('|')[1])
                        print('.............................')

                        #print(seq_t.id.split('|')[1])
                        #print('length: ' + str(min(len(seq_q.seq), len(seq_t.seq))))
                        #print('length2: ' + str(fn_format_alignment(*align[0])[4]))
                        #print('Score: ' + str(fn_format_alignment(*align[0])[0]))
                        #print('matches: ' + str(fn_format_alignment(*align[0])[1]))
                        #print('mismatches: ' + str(fn_format_alignment(*align[0])[2]))
                        #print('mismatches+ : '+ str(mismatch_positive))
                        #print('gaps: ' + str(fn_format_alignment(*align[0])[3]))

                        #print('Identity: ' + str(Identity))

                        #print('Normalize_Score: ' + str(Normalize_Score))
                        #print('Similarity: ' + str(Similarity))



                        #print('.............................')

                        #print(align[0])
                        #print('---------------------------------------')
                        #print(align_score)
                        #print(align_score[1])
                        #print(align_score[2])
                        #print(align_score[3])
                        #print(type(align_score))  
                            
                    except Exception as e:
                        print(e)
                        continue # doing nothing on exception
            #count+=1
            #print(count)
            #print(seq_q.id.split('|')[1] + '  Completed')
            except Exception as e:
                    print(e)
                    continue # doing nothing on exception

    print("Task Completed.....")






In [10]:
print("started....")
calculate_similarity(query_seq,target_seq)
print("completed.....")


started....
started task...
1
Q07955
.............................
2
Q07955
.............................
3
Q07955
.............................
4
Q07955
.............................
5
Q07955
.............................
6
Q07955
.............................
7
Q07955
.............................
8
Q07955
.............................
9
Q07955
.............................
10
Q07955
.............................
11
Q07955
.............................
12
Q07955
.............................
13
Q07955
.............................
14
Q07955
.............................
15
Q07955
.............................
16
Q07955
.............................
17
Q07955
.............................
18
Q07955
.............................
19
Q07955
.............................
20
Q07955
.............................
21
Q07955
.............................
22
Q07955
.............................
23
Q07955
.............................
24
Q07955
.............................
25
Q07955
...........

203
Q9NZI8
.............................
204
Q9NZI8
.............................
205
Q9NZI8
.............................
206
Q9NZI8
.............................
207
Q9NZI8
.............................
208
Q9NZI8
.............................
209
Q9NZI8
.............................
210
Q9NZI8
.............................
211
Q9NZI8
.............................
212
Q9NZI8
.............................
213
Q9NZI8
.............................
214
Q9NZI8
.............................
215
Q9NZI8
.............................
216
Q9NZI8
.............................
217
Q9NZI8
.............................
218
Q9NZI8
.............................
219
Q9NZI8
.............................
220
Q9NZI8
.............................
221
Q9NZI8
.............................
222
Q9NZI8
.............................
223
Q9NZI8
.............................
224
Q9NZI8
.............................
225
Q9NZI8
.............................
226
P26599
.............................
227
P26599
.....

403
Q13148
.............................
404
Q13148
.............................
405
Q13148
.............................
406
Q13148
.............................
407
Q13148
.............................
408
Q13148
.............................
409
Q13148
.............................
410
Q13148
.............................
411
Q13148
.............................
412
Q13148
.............................
413
Q13148
.............................
414
Q13148
.............................
415
Q13148
.............................
416
Q13148
.............................
417
Q13148
.............................
418
Q13148
.............................
419
Q13148
.............................
420
Q13148
.............................
421
Q13148
.............................
422
Q13148
.............................
423
Q13148
.............................
424
Q13148
.............................
425
Q13148
.............................
426
Q9Y6M1
.............................
427
Q9Y6M1
.....

603
Q6PJX3
.............................
604
Q6PJX3
.............................
605
Q6PJX3
.............................
606
Q6PJX3
.............................
607
Q6PJX3
.............................
608
Q6PJX3
.............................
609
Q6PJX3
.............................
610
Q6PJX3
.............................
611
Q6PJX3
.............................
612
Q6PJX3
.............................
613
Q6PJX3
.............................
614
Q6PJX3
.............................
615
Q6PJX3
.............................
616
Q6PJX3
.............................
617
Q6PJX3
.............................
618
Q6PJX3
.............................
619
Q6PJX3
.............................
620
Q6PJX3
.............................
621
Q6PJX3
.............................
622
Q6PJX3
.............................
623
Q6PJX3
.............................
624
Q6PJX3
.............................
625
Q6PJX3
.............................
Task Completed.....
completed.....


In [ ]:

#blosum62.update(((b,a),val) for (a,b),val in blosum62.items())
#blosum62


In [ ]:
#aligner = Align.PairwiseAligner()
#aligner.substitution_matrix = blosum62
#aligner.mode = 'local'
#aligner.open_gap_score = -10
#aligner.extend_gap_score = -0.5
#alignments = alignelr.align(query_seq[0].seq, target_seq[0].seq)

#print(alignments[0].score)
#print(aligner)

In [6]:
df=pd.read_csv('../Data/NpInter2 Interacting Protiens/Protein-Protein Sequence Similarity/Protien-Protien_Similarity.csv')
df.shape

(20880, 6)

In [7]:
df.nunique()

Query_Seq_ID        145
Target_Seq_ID       145
Align_Score         399
Normalize_Score      45
Identity           1397
Similarity         1843
dtype: int64

In [8]:
df_pro=df[df['Align_Score']>=30]

df_pro.shape

(19384, 6)

In [9]:
df_pro.nunique()

Query_Seq_ID        145
Target_Seq_ID       145
Align_Score         374
Normalize_Score      45
Identity           1390
Similarity         1840
dtype: int64

In [10]:
df_pro['Align_Score'].max()

3900.5

In [11]:
df_pro.head()

,Query_Seq_ID,Target_Seq_ID,Align_Score,Normalize_Score,Identity,Similarity
0,Q9UPQ9,Q13148,118.5,0.02,24.11%,34.11%
1,Q9UPQ9,P08579,73.0,0.02,24.76%,36.19%
2,Q9UPQ9,O14746,81.5,0.01,21.54%,31.49%
3,Q9UPQ9,Q9UHB9,59.0,0.01,30.00%,51.66%
4,Q9UPQ9,O75940,67.0,0.01,21.66%,31.25%


In [4]:
from Bio._py3k import StringIO

alignments = list(EmbossIterator(StringIO(pair_example)))
assert len(alignments) == 1
assert len(alignments[0]) == 2
assert [r.id for r in alignments[0]] \
       == ["IXI_234", "IXI_235"]

alignments = list(EmbossIterator(StringIO(simple_example)))
assert len(alignments) == 1
assert len(alignments[0]) == 4
assert [r.id for r in alignments[0]] \
       == ["IXI_234", "IXI_235", "IXI_236", "IXI_237"]

alignments = list(EmbossIterator(StringIO(pair_example + simple_example)))
assert len(alignments) == 2
assert len(alignments[0]) == 2
assert len(alignments[1]) == 4
assert [r.id for r in alignments[0]] \
       == ["IXI_234", "IXI_235"]
assert [r.id for r in alignments[1]] \
       == ["IXI_234", "IXI_235", "IXI_236", "IXI_237"]

alignments = list(EmbossIterator(StringIO(pair_example2)))
assert len(alignments) == 5
assert len(alignments[0]) == 2
assert [r.id for r in alignments[0]] \
       == ["ref_rec", "gi|94968718|receiver"]
assert [r.id for r in alignments[4]] \
       == ["ref_rec", "gi|94970041|receiver"]

alignments = list(EmbossIterator(StringIO(pair_example3)))
assert len(alignments) == 1
assert len(alignments[0]) == 2
assert [r.id for r in alignments[0]] \
       == ["asis", "asis"]

print("Done")

NameError: name 'EmbossIterator' is not defined